In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import numpy as np
import joblib

In [2]:
df = pd.read_csv("../Data/FinalData.csv", encoding='utf8')
df.head(5)


,Date,co,no2,o3,pm10,pm25,so2,Temp,Rain,Cloud,Pressure,Wind,Gust
0,2015-01-01,28.00,40.00,22.00,124.00,194.67,6.00,17.50,0.00,0.00,1023.50,7.12,12.62
1,2015-01-02,43.00,39.00,69.00,144.00,186.00,8.00,17.38,0.00,5.88,1022.38,6.25,9.12
2,2015-01-03,28.00,25.00,24.00,151.00,227.00,12.00,17.00,0.00,41.25,1019.50,5.75,8.50
3,2015-01-04,15.07,18.24,10.07,91.05,253.00,12.76,19.62,0.01,62.38,1013.50,7.62,13.75
4,2015-01-05,33.00,35.00,65.00,115.00,160.29,5.00,21.88,0.00,64.00,1010.62,6.75,10.12


In [3]:
lag_steps = 12
for lag in range(1, lag_steps + 1):
    df[f'co_t-{lag}'] = df['co'].shift(lag)
    df[f'no2_t-{lag}'] = df['no2'].shift(lag)
    df[f'o3_t-{lag}'] = df['o3'].shift(lag)
    df[f'pm10_t-{lag}'] = df['pm10'].shift(lag)
    df[f'pm25_t-{lag}'] = df['pm25'].shift(lag)
    df[f'so2_t-{lag}'] = df['so2'].shift(lag)
    df[f'Temp_t-{lag}'] = df['Temp'].shift(lag)
    df[f'Rain_t-{lag}'] = df['Rain'].shift(lag)
    df[f'Cloud_t-{lag}'] = df['Cloud'].shift(lag)
    df[f'Pressure_t-{lag}'] = df['Pressure'].shift(lag)
    df[f'Wind_t-{lag}'] = df['Wind'].shift(lag)
    df[f'Gust_t-{lag}'] = df['Gust'].shift(lag)

    # Bỏ qua các hàng có giá trị NaN
df = df[lag_steps:-1].reset_index(drop=True)


C:\Users\namkh\AppData\Local\Temp\ipykernel_17616\1935899651.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'o3_t-{lag}'] = df['o3'].shift(lag)
C:\Users\namkh\AppData\Local\Temp\ipykernel_17616\1935899651.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'pm10_t-{lag}'] = df['pm10'].shift(lag)
C:\Users\namkh\AppData\Local\Temp\ipykernel_17616\1935899651.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider j

In [4]:
features = []
for col in ['co','no2','o3','pm10','pm25','so2','Temp','Rain','Cloud','Pressure','Wind','Gust']:
    for lag in range(1, lag_steps + 1):
        features.append(f'{col}_t-{lag}')

X = df[features]
y = df[['co','no2','o3','pm10','pm25','so2','Temp','Rain','Cloud','Pressure','Wind','Gust']]

In [5]:
X.head()

,co_t-1,co_t-2,co_t-3,co_t-4,co_t-5,co_t-6,co_t-7,co_t-8,co_t-9,co_t-10,...,Gust_t-3,Gust_t-4,Gust_t-5,Gust_t-6,Gust_t-7,Gust_t-8,Gust_t-9,Gust_t-10,Gust_t-11,Gust_t-12
0,17.0,20.0,25.0,32.0,22.0,27.0,27.0,33.0,15.07,28.00,...,15.12,14.12,11.50,17.62,10.75,10.12,13.75,8.50,9.12,12.62
1,21.0,17.0,20.0,25.0,32.0,22.0,27.0,27.0,33.00,15.07,...,12.50,15.12,14.12,11.50,17.62,10.75,10.12,13.75,8.50,9.12
2,22.0,21.0,17.0,20.0,25.0,32.0,22.0,27.0,27.00,33.00,...,13.88,12.50,15.12,14.12,11.50,17.62,10.75,10.12,13.75,8.50
3,23.0,22.0,21.0,17.0,20.0,25.0,32.0,22.0,27.00,27.00,...,9.62,13.88,12.50,15.12,14.12,11.50,17.62,10.75,10.12,13.75
4,28.0,23.0,22.0,21.0,17.0,20.0,25.0,32.0,22.00,27.00,...,6.50,9.62,13.88,12.50,15.12,14.12,11.50,17.62,10.75,10.12


In [10]:
tscv = TimeSeriesSplit(n_splits=5)

param_grid = {
    'n_estimators': [50, 200, 500],
    'max_depth': [None, 20, 50, 100],
    'max_features': ['sqrt', 'log2'],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False],
}

rf = RandomForestRegressor(random_state=42)

grid = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=tscv,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1
)

grid.fit(X, y)

print("Best params:", grid.best_params_)
print("Best CV RMSE:", -grid.best_score_)

Best params: {'bootstrap': False, 'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 500}
Best CV RMSE: 9.126865818385557


In [11]:
best_rf = grid.best_estimator_
X1 = X.iloc[[-1]]
y1 = best_rf.predict(X1)
np.set_printoptions(suppress=True, precision=2)
print(np.round(y1, 2))

[[   6.43   18.87   15.98   51.52   45.27    8.09   25.03    0.57   58.39
  1010.03   11.45   18.64]]


In [ ]:
joblib.dump(best_rf, './Built_Model/random_forests_model.pkl')

loaded_model = joblib.load('./Built_Model/random_forests_model.pkl')